In [17]:
# poster-preview.ipynb (Tweet formatter notebook with Claude + static + dynamic hashtags)

import json
import os
import re
import textwrap
import csv

# 🧭 Robust project root resolver
try:
    PROJECT_ROOT = os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
except NameError:
    PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))

SUMMARY_PATH = os.path.join(PROJECT_ROOT, "summarized_output.json")
EXPORT_JSON = os.path.join(PROJECT_ROOT, "tweet_threads.json")
EXPORT_CSV = os.path.join(PROJECT_ROOT, "tweet_threads.csv")

DEFAULT_HASHTAGS = ["#AI"]
MAX_TWEET_LENGTH = 280

# Load summaries
try:
    with open(SUMMARY_PATH, "r", encoding="utf-8") as f:
        articles = json.load(f)
except FileNotFoundError:
    print(f"[ERROR] summarized_output.json not found at: {SUMMARY_PATH}")
    articles = []

# Sentence splitting regex
def split_sentences(text):
    return re.split(r'(?<=[.!?]) +', text.strip())

# Chunk summary into tweetable parts
def chunk_summary_into_thread(summary, title, url, hashtags=DEFAULT_HASHTAGS):
    sentences = split_sentences(summary)
    thread = []
    current = ""

    for sentence in sentences:
        if len(current) + len(sentence) + 1 <= MAX_TWEET_LENGTH:
            current += (" " if current else "") + sentence
        else:
            thread.append(current.strip())
            current = sentence
    if current:
        thread.append(current.strip())

    # Add title to first tweet
    if title:
        thread[0] = f"{title}\n{thread[0]}"

    # Final tweet: link + hashtags
    hashtag_block = " ".join(hashtags)
    link_tweet = f"{url}\n{hashtag_block}".strip()
    if len(link_tweet) > MAX_TWEET_LENGTH:
        hashtag_block = "#AI"
        link_tweet = f"{url}\n{hashtag_block}"
    thread.append(link_tweet)

    return thread

# Format + export
all_threads = []

for i, article in enumerate(articles[:5]):
    title = article.get("title", "")
    summary = article.get("v1_summary", "")
    url = article.get("url", "")
    dyn_tags = [tag for tag in article.get("hashtags", "").split() if tag.startswith("#")]
    hashtags = list(dict.fromkeys(DEFAULT_HASHTAGS + dyn_tags[:3]))

    thread = chunk_summary_into_thread(summary, title, url, hashtags)

    for j, tweet in enumerate(thread):
        all_threads.append({
            "article_index": i + 1,
            "thread_index": j + 1,
            "tweet": tweet,
            "characters": len(tweet),
            "url": url,
            "hashtags": " ".join(hashtags)
        })

    print(f"\n🧵 === Thread for Article {i+1} ===")
    for j, tweet in enumerate(thread):
        print(f"\n--- Tweet {j+1} ---\n{tweet}\nCharacters: {len(tweet)}")

# Save outputs
with open(EXPORT_JSON, "w", encoding="utf-8") as f:
    json.dump(all_threads, f, indent=2, ensure_ascii=False)

with open(EXPORT_CSV, "w", encoding="utf-8", newline='') as f:
    writer = csv.DictWriter(f, fieldnames=all_threads[0].keys())
    writer.writeheader()
    writer.writerows(all_threads)

print(f"\n✅ Exported {len(all_threads)} tweets across {len(articles)} threads to:")
print(f"- {EXPORT_JSON}")
print(f"- {EXPORT_CSV}")



🧵 === Thread for Article 1 ===

--- Tweet 1 ---
Sculpting Subspaces: Constrained Full Fine-Tuning in LLMs for Continual Learning
🧠✨ Exciting AI breakthrough alert! 🚀 Scientists have cracked the code on making language models learn new tricks without forgetting old ones. It's like teaching an old dog new tricks, but the dog remembers ALL its tricks!
Characters: 302

--- Tweet 2 ---
🐶🎓

This clever method, using some fancy math called "adaptive SVD," lets AI models keep learning without getting brain freeze. 🧊🤯 The result? AI that's smarter, more flexible, and doesn't need a gazillion extra parts to keep growing. Now that's what we call a win-win! 🏆🎉
Characters: 272

--- Tweet 3 ---
https://arxiv.org/abs/2504.07097
#AI #ContinualLearning #LLM #CatastrophicForgetting
Characters: 84

🧵 === Thread for Article 2 ===

--- Tweet 1 ---
Are We Done with Object-Centric Learning?
🤖📸 Object-centric learning just got a major upgrade! Researchers have found a way to separate objects in images more e